<a href="https://colab.research.google.com/github/zhulkhoirr/Steam-Recommendation-System/blob/main/SteamRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistem Rekomendasi Game Pada Platform Steam

## 1. Download Datasets Dari Kaggle

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"zrifat","key":"233abd402bb5ca54e6ae1d6829961cef"}'}

In [2]:
!pip install -q kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d antonkozyriev/game-recommendations-on-steam

Dataset URL: https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam
License(s): CC0-1.0
100% 657M/660M [00:31<00:00, 26.3MB/s]
100% 660M/660M [00:31<00:00, 22.1MB/s]


In [ ]:
!unzip -q model.zip

In [6]:
!unzip -q game-recommendations-on-steam.zip

## 2. Memahami Data

In [143]:
import numpy as np
import pandas as pd

In [144]:
recommendations_df = pd.read_csv('recommendations.csv')

recommendations_df.sample(5)

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
34861524,265550,0,0,2018-05-20,True,19.5,5201324,34861524
29514139,1209040,0,0,2020-08-19,True,4.4,1909802,29514139
6187416,1066890,4,0,2021-10-02,True,246.1,1899750,6187416
16530915,847370,0,0,2022-08-03,True,70.8,1073346,16530915
2286207,1229490,3,6,2022-08-26,False,8.0,3953625,2286207


In [145]:
games_df = pd.read_csv('games.csv')

games_df.sample(5)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
40124,924370,Kontrakt,2018-10-22,True,False,True,Mixed,57,19,14.99,14.99,0.0,True
7845,1925320,Escape Memoirs: Mansion Heist,2022-06-16,True,False,False,Very Positive,89,129,7.99,7.99,0.0,True
35606,640800,Skyfront VR,2019-03-08,True,False,False,Mostly Positive,71,105,9.99,9.99,0.0,True
23470,1583720,Entropy : Zero 2,2022-08-20,True,False,False,Overwhelmingly Positive,98,14612,0.00,0.00,0.0,True
45989,1049000,Chronoraptor,2019-04-29,True,False,False,Positive,93,15,0.00,0.00,0.0,True


In [146]:
sorted_df = games_df.sort_values(by='user_reviews', ascending=False).head(10)

sorted_df

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
14398,730,Counter-Strike: Global Offensive,2012-08-21,True,True,True,Very Positive,88,7494460,15.0,0.0,0.0,True
47770,578080,PUBG: BATTLEGROUNDS,2017-12-21,True,False,False,Mixed,57,2217226,0.0,0.0,0.0,True
13176,570,Dota 2,2013-07-09,True,True,True,Very Positive,82,2045628,0.0,0.0,0.0,True
12717,271590,Grand Theft Auto V,2015-04-13,True,False,False,Very Positive,86,1484122,0.0,0.0,0.0,True
14535,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,True,False,False,Very Positive,86,993312,20.0,0.0,0.0,True
47380,440,Team Fortress 2,2007-10-10,True,True,True,Very Positive,93,985819,0.0,0.0,0.0,True
13035,105600,Terraria,2011-05-16,True,True,True,Overwhelmingly Positive,97,943413,10.0,0.0,0.0,True
15363,4000,Garry's Mod,2006-11-29,True,True,True,Overwhelmingly Positive,96,853733,10.0,0.0,0.0,True
13173,252490,Rust,2018-02-08,True,True,False,Very Positive,87,786668,40.0,0.0,0.0,True
14376,1172470,Apex Legends™,2020-11-04,True,False,False,Very Positive,80,713182,0.0,0.0,0.0,True


In [147]:
import json

with open('games_metadata.json', 'r') as file:
    data = file.read()

json_objects = data.split('\n')

data_list = []
for obj in json_objects:
    if obj.strip():
        data_list.append(json.loads(obj))

games_metadata_df = pd.DataFrame(data_list)
games_metadata_df['tags'] = games_metadata_df['tags'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
games_metadata_df.sample(5)


,app_id,description,tags
44696,593400,Float Gallery shows the work of artists workin...,"Free to Play, Casual, Simulation, VR"
6220,492500,After countless days and nights celebrating wi...,"RPG, Dungeon Crawler, Action, Grid-Based Movem..."
23456,1215500,"The year is 2087, the Technotron rules the wor...","Casual, Action, Indie, Adventure, Simulation, ..."
37787,679300,"In PATHOS, a resilient mind is strangled by tr...","Sokoban, Puzzle, Difficult, 2D, Relaxing, Stor..."
36043,2121680,Steamcore is a MMO PvPvE shooter where players...,"Early Access, Action, eSports, Shooter, FPS, L..."


In [148]:
gg = games_metadata_df[games_metadata_df['app_id'] == 409710]

gg

,app_id,description,tags
25126,409710,"BioShock is a shooter unlike any other, loaded...","FPS, Story Rich, Atmospheric, Horror, Singlepl..."


In [149]:
games_with_metadata_df = pd.merge(games_df, games_metadata_df, on='app_id', how='outer')

selected_columns = ['app_id', 'title', 'rating', 'user_reviews', 'price_final', 'description', 'tags']

games_with_metadata_df = games_with_metadata_df[selected_columns]

games_with_metadata_df.sample(5)

,app_id,title,rating,user_reviews,price_final,description,tags
14553,1166990,Kukoro: Stream chat games,Overwhelmingly Positive,506,4.99,Kukoro: Stream chat games is the ultimate mini...,"Casual, Experimental, Arcade, Collectathon, 2D..."
20554,402340,Blue Rider,Positive,46,9.99,"In a valley flooded with bullets, your next ch...","Action, Indie, Shoot 'Em Up, Arcade, Shooter, ..."
11068,1804500,Z-Collapse,Mostly Positive,57,12.99,"Inspired by Max Brooks' World War Z novel, Z-C...","Simulation, Strategy, Grand Strategy, RTS, Out..."
10393,13620,Tom Clancy's Ghost Recon® Desert Siege™,Very Positive,93,4.99,"East Africa, 2009. A 60-year conflict boils ov...","Action, Shooter, FPS, First-Person, Tactical, ..."
19253,325968,Train Simulator: NJ TRANSIT® F40PH -2CAT Loco ...,Positive,16,19.99,,Simulation


In [150]:
filtered_df = games_with_metadata_df[selected_columns]

sorted_df = filtered_df.sort_values(by='user_reviews', ascending=False).head(10)

sorted_df

,app_id,title,rating,user_reviews,price_final,description,tags
14398,730,Counter-Strike: Global Offensive,Very Positive,7494460,15.0,,
47770,578080,PUBG: BATTLEGROUNDS,Mixed,2217226,0.0,,
13176,570,Dota 2,Very Positive,2045628,0.0,,
12717,271590,Grand Theft Auto V,Very Positive,1484122,0.0,,
14535,359550,Tom Clancy's Rainbow Six® Siege,Very Positive,993312,20.0,,
47380,440,Team Fortress 2,Very Positive,985819,0.0,,
13035,105600,Terraria,Overwhelmingly Positive,943413,10.0,,
15363,4000,Garry's Mod,Overwhelmingly Positive,853733,10.0,,
13173,252490,Rust,Very Positive,786668,40.0,,
14376,1172470,Apex Legends™,Very Positive,713182,0.0,,


In [151]:
users_df = pd.read_csv('users.csv')

users_df.sample(5)

,user_id,products,reviews
13731433,3701358,28,1
2437119,8102910,137,1
11809003,11341425,146,1
8936451,2545637,9,6
2371330,11872142,132,9


# Neural Collaborative Filtering

## 1. Mempersiapkan Data

In [152]:
recommendations_df = pd.read_csv('recommendations.csv', usecols=['app_id', 'is_recommended', 'user_id', 'hours'])

recommendations_df.head(5)

,app_id,is_recommended,hours,user_id
0,975370,True,36.3,51580
1,304390,False,11.5,2586
2,1085660,True,336.5,253880
3,703080,True,27.4,259432
4,526870,True,7.9,23869


In [153]:
hours_count = recommendations_df[recommendations_df['hours'] == 0.0].count()

print(f"Jumlah nilai 0.0 di kolom 'hours': {hours_count['hours']}")

Jumlah nilai 0.0 di kolom 'hours': 176329


In [154]:
recommendations_df = recommendations_df[recommendations_df['hours'] != 0.0]

In [155]:
zero_hours_count = (recommendations_df['hours'] == 0.0).sum()

print(f"Jumlah nilai 0.0 di kolom 'hours': {zero_hours_count}")

Jumlah nilai 0.0 di kolom 'hours': 0


In [156]:
n_recommendations = recommendations_df['user_id'].value_counts()
print(n_recommendations)

user_id
11764552    5652
5112758     3657
11656130    3648
5669734     3479
11553593    3118
            ... 
5586939        1
4497103        1
2719342        1
7185639        1
2306695        1
Name: count, Length: 13760373, dtype: int64


In [157]:
n_recommendations = recommendations_df['user_id'].value_counts()
recommendations_df = recommendations_df[recommendations_df['user_id'].isin(n_recommendations[n_recommendations >=50].index)].copy()
recommendations_df.head(5)

,app_id,is_recommended,hours,user_id
58,602960,True,41.9,737481
138,570940,False,73.3,5136277
151,1196590,True,19.0,5935019
203,304390,False,42.5,10567745
209,244850,True,540.7,11143713


In [158]:
print("Jumlah duplikasi:",  recommendations_df.duplicated().sum())

Jumlah duplikasi: 1


In [159]:
recommendations_df.drop_duplicates(inplace=True)
print("Jumlah duplikasi:",  recommendations_df.duplicated().sum())

Jumlah duplikasi: 0


In [160]:
recommendations_df.isnull().sum()

app_id            0
is_recommended    0
hours             0
user_id           0
dtype: int64

In [161]:
recommendations_df['is_recommended'] = recommendations_df['is_recommended'].astype(int)

## 2. Preprocessing Data

In [162]:
user_ids = recommendations_df['user_id'].unique().tolist()
user2user_encoded = {x:i for i, x in enumerate(user_ids)}
user_encoded2user = {i:x for i, x in enumerate(user_ids)}
recommendations_df['user'] = recommendations_df['user_id'].map(user2user_encoded)
n_users = len(user2user_encoded)

app_ids = recommendations_df['app_id'].unique().tolist()
app2app_encoded = {x:i for i, x in enumerate(app_ids)}
app_encoded2app = {i:x for i, x in enumerate(app_ids)}
recommendations_df['app'] = recommendations_df['app_id'].map(app2app_encoded)
n_app  = len(app2app_encoded)

print("Jumlah user: {}, Jumlah game: {}".format(n_users, n_app))

Jumlah user: 36646, Jumlah game: 36028


In [163]:
recommendations_df

,app_id,is_recommended,hours,user_id,user,app
58,602960,1,41.9,737481,0,0
138,570940,0,73.3,5136277,1,1
151,1196590,1,19.0,5935019,2,2
203,304390,0,42.5,10567745,3,3
209,244850,1,540.7,11143713,4,4
...,...,...,...,...,...,...
41154764,1818450,1,1.0,13261742,24873,991
41154770,700600,1,2.0,6382076,35046,2837
41154771,362960,1,35.0,7858713,21437,3024
41154784,633230,0,20.0,8107668,18148,2391


In [164]:
recommendations_df = recommendations_df.sample(frac=1, random_state=46)

X = recommendations_df[['user', 'app']].values
y = recommendations_df['is_recommended']

In [165]:
X

array([[ 8127,    52],
       [ 5147, 27858],
       [ 6385,  2674],
       ...,
       [30719,  1078],
       [ 7675,   401],
       [23746, 25712]])

In [166]:
y

13131890    1
35625112    1
16695870    1
14523913    1
6006244     1
           ..
3402980     0
36455208    1
9328930     0
19537234    1
36598476    0
Name: is_recommended, Length: 3376881, dtype: int64

In [167]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

print("Train set is_recommended: {}".format(len(y_train)))
print("Test set is_recommended: {}".format(len(y_test)))

Train set is_recommended: 2701504
Test set is_recommended: 675377


In [168]:
X_train

array([[29438,  3890],
       [ 3727, 23255],
       [   55,   745],
       ...,
       [  991, 27745],
       [35784, 27875],
       [25174,  1776]])

In [169]:
X_train[:, 0]

array([29438,  3727,    55, ...,   991, 35784, 25174])

In [170]:
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [171]:
X_train_array

[array([29438,  3727,    55, ...,   991, 35784, 25174]),
 array([ 3890, 23255,   745, ..., 27745, 27875,  1776])]

## 3. Membuat Model

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout, Dot, BatchNormalization

def RecommenderNet(n_users, n_app, embedding_size=32):
  user_input = Input(shape=(1), name='user')
  user_embedding = Embedding(n_users, embedding_size, name='user_embedding')(user_input)
  app_input = Input(shape=(1), name='app')
  app_embedding = Embedding(n_app, embedding_size, name='app_embedding')(app_input)

  # concatenate = Concatenate(name='concatenate')([user_vec, app_vec])
  dot = Dot(axes=1, normalize=True, name='dot')([user_embedding, app_embedding])
  flatten = Flatten(name='flatten')(dot)

  dense = Dense(128, activation='relu', name='dense')(flatten)
  dropout = Dropout(0.4, name='dropout')(dense)
  batch_norm = BatchNormalization(name='batch_norm')(dropout)
  output = Dense(1, activation='sigmoid', name='output')(batch_norm)

  model = Model([user_input, app_input], output)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mae', 'mse'])

  return model

model = RecommenderNet(n_users, n_app)

model.summary()

In [ ]:
history = model.fit(
    x=X_train_array,
    y=y_train,
    epochs=15,
    batch_size=256,
    validation_data=(X_test_array, y_test)
)

In [ ]:
loss, mae, mse = model.evaluate(X_test_array, y_test)

print(f'Loss: {loss}, MAE: {mae}, MSE: {mse}')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
model.save('model/model.h5')

## 4. Prediksi Rekomendasi

In [172]:
print('Pilih user random')

recommendation_per_user = recommendations_df.groupby('user_id').size()

random_user =  recommendation_per_user[recommendation_per_user > 50].sample(1).index[0]

print(f'user_id: {random_user}')

Pilih user random
user_id: 6429716


In [173]:
from tensorflow.keras.models import load_model

model = load_model('model.h5')

print(f'Game yang direkomendasikan untuk user: {random_user}')
print('===' * 25)

game_played_by_user = recommendations_df[recommendations_df.user_id == random_user]
game_not_played_df = games_with_metadata_df[~games_with_metadata_df['app_id'].isin(game_played_by_user.app_id.values)]
game_not_played = list(set(game_not_played_df['app_id']).intersection(set(app2app_encoded.keys())))
game_not_played = [[app2app_encoded.get(x)] for x in game_not_played if x in app2app_encoded]

user_encoder = user2user_encoded.get(random_user)
user_game_array = np.hstack(([[user_encoder]] * len(game_not_played), game_not_played))
user_game_array = [user_game_array[:, 0], user_game_array[:, 1]]

recommendations = model.predict(user_game_array).flatten()
top_recommendations = (-recommendations).argsort()[:10]

recommended_games_ids = [app_encoded2app.get(game_not_played[top_recommendations[x]][0]) for x in range(len(top_recommendations)) if game_not_played[top_recommendations[x]][0] in app_encoded2app]

results = []
top_recommendations_ids = []

for index, app_id in enumerate(game_not_played):
  recommendation = recommendations[index]
  id_ = app_encoded2app.get(app_id[0])

  if id_ in recommended_games_ids:
    top_recommendations_ids.append(id_)
    try:
      condition = (games_with_metadata_df.app_id == id_)
      if games_with_metadata_df[condition].empty:
        print(f"No metadata found for game ID {id_}")
        continue

      title = games_with_metadata_df[condition]['title'].values[0]
      rating = games_with_metadata_df[condition]['rating'].values[0]
      user_reviews = games_with_metadata_df[condition]['user_reviews'].values[0]
      price_final = games_with_metadata_df[condition]['price_final'].values[0]
      description = games_with_metadata_df[condition]['description'].values[0]
      tags = games_with_metadata_df[condition]['tags'].values[0]

      results.append({
        'title': title,
        'pred_recommendations': recommendation,
        'rating': rating,
        'user_reviews': user_reviews,
        'price_final': price_final,
        'description': description,
        'tags': tags
      })
    except Exception as e:
        print(f"Error processing game {id_}: {e}")
        continue

print("---" * 25)
print("> Top 10 game recommendations")
print("---" * 25)

results_df = pd.DataFrame(results)

results_df = results_df.sort_values(by='pred_recommendations', ascending=False)

results_df

Game yang direkomendasikan untuk user: 6429716
1124/1124 [==============================] - 3s 2ms/step
---------------------------------------------------------------------------
> Top 10 game recommendations
---------------------------------------------------------------------------


,title,pred_recommendations,rating,user_reviews,price_final,description,tags
9,Zup! X,0.989777,Overwhelmingly Positive,5785,1.99,Minimal physical puzzle with explosions,"Puzzle, Casual, Indie, Fast-Paced, 2D, Physics..."
5,Romance of the Three Kingdoms VII with Power U...,0.988798,Mostly Positive,88,19.99,"""Romance of the Three Kingdoms Ⅶ with Power Up...",Strategy
1,ULTRAKILL,0.988500,Overwhelmingly Positive,67817,25.00,,
4,The King's Request: Physiology and Anatomy Rev...,0.986026,Very Positive,159,0.00,This short adventure game uses your knowledge ...,"Adventure, RPG, Free to Play, Casual, RPGMaker..."
0,Wallpaper Engine,0.985945,Overwhelmingly Positive,637341,4.00,,
8,Totally Accurate Battle Simulator,0.985723,Overwhelmingly Positive,104218,20.00,,
3,Thirty Years' War,0.985646,Mixed,35,29.99,Thirty Years War is a new game designed by dev...,"Strategy, Simulation, Historical, Wargame, Gra..."
2,Escape : Lia,0.985629,Mostly Negative,26,0.00,Here's a girl named Lia -- an 8-year-old child...,"Strategy, Indie, Singleplayer, First-Person, E..."
6,The Agency of Anomalies: Mystic Hospital Colle...,0.985007,Positive,14,4.99,Explore haunted military hospitals!,"Adventure, Casual, Hidden Object"
7,Celeste,0.984956,Overwhelmingly Positive,72238,20.00,,


# Cosine Similarity (Content-Based Filtering)

## Seleksi Fitur

In [174]:
selected_columns = ['title', 'description', 'tags']

games_with_metadata_df = games_with_metadata_df[selected_columns]

In [175]:
games_with_metadata_df

,title,description,tags
0,Prince of Persia: Warrior Within™,Enter the dark underworld of Prince of Persia ...,"Action, Adventure, Parkour, Third Person, Grea..."
1,BRINK: Agents of Change,,Action
2,Monaco: What's Yours Is Mine,Monaco: What's Yours Is Mine is a single playe...,"Co-op, Stealth, Indie, Heist, Local Co-Op, Str..."
3,Escape Dead Island,Escape Dead Island is a Survival-Mystery adven...,"Zombies, Adventure, Survival, Action, Third Pe..."
4,Dungeon of the ENDLESS™,Dungeon of the Endless is a Rogue-Like Dungeon...,"Roguelike, Strategy, Tower Defense, Pixel Grap..."
...,...,...,...
50867,I Expect You To Die 3: Cog in the Machine,,
50868,PAYDAY 3,,
50869,Eternights,,
50870,Forgive Me Father 2,Embark on a journey into the darkest nightmare...,"Early Access, FPS, Action, Retro, First-Person..."


In [176]:
print(f"Isi datanya ada {games_with_metadata_df.shape[0]} baris, dan {games_with_metadata_df.shape[1]} kolom")

Isi datanya ada 50872 baris, dan 3 kolom


In [177]:
games_with_metadata_df.isnull().sum()

title          0
description    0
tags           0
dtype: int64

In [178]:
features_combine = games_with_metadata_df['title'] + ' ' + games_with_metadata_df['description'] + ' ' + games_with_metadata_df['tags']

features_combine

0        Prince of Persia: Warrior Within™ Enter the da...
1                          BRINK: Agents of Change  Action
2        Monaco: What's Yours Is Mine Monaco: What's Yo...
3        Escape Dead Island Escape Dead Island is a Sur...
4        Dungeon of the ENDLESS™ Dungeon of the Endless...
                               ...                        
50867          I Expect You To Die 3: Cog in the Machine  
50868                                           PAYDAY 3  
50869                                         Eternights  
50870    Forgive Me Father 2 Embark on a journey into t...
50871                                          FatalZone  
Length: 50872, dtype: object

In [179]:
print('Data type: {}'.format(type(features_combine)))

Data type: <class 'pandas.core.series.Series'>


In [180]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

In [181]:
feature_vector = vectorizer.fit_transform(features_combine)

In [182]:
print(feature_vector)

  (0, 35645)	0.04655948366636258
  (0, 10029)	0.06468808171186599
  (0, 50147)	0.04330869784127758
  (0, 31854)	0.05837487966319662
  (0, 37809)	0.0483463834470858
  (0, 43162)	0.04598690740000512
  (0, 16251)	0.10323639686073102
  (0, 19133)	0.06267791996110231
  (0, 27379)	0.10217514035048705
  (0, 41166)	0.07478941908122651
  (0, 19962)	0.07453290659158938
  (0, 8913)	0.06472333790781681
  (0, 3998)	0.045912664205036764
  (0, 46343)	0.08715210187124423
  (0, 34074)	0.05171213177215122
  (0, 40866)	0.030140858049291626
  (0, 41920)	0.05458382522093901
  (0, 19403)	0.05692191041434069
  (0, 33419)	0.04526445247821519
  (0, 32855)	0.09179661885137286
  (0, 1783)	0.030632485984244416
  (0, 1578)	0.02963903278018437
  (0, 11687)	0.07545976129110635
  (0, 34818)	0.18544540182605906
  (0, 11974)	0.13601497749378277
  :	:
  (50870, 11647)	0.15027084710636365
  (50870, 49419)	0.12437438858294529
  (50870, 17545)	0.20087753480841658
  (50870, 30760)	0.1850690413377434
  (50870, 48594)	0.09568

In [183]:
print(vectorizer.vocabulary_)

{'prince': 34946, 'persia': 33409, 'warrior': 49244, 'enter': 15051, 'dark': 11428, 'underworld': 47344, 'sword': 44154, 'slashing': 41171, 'sequel': 39914, 'critically': 10657, 'acclaimed': 1457, 'sands': 39010, 'time': 45575, 'hunted': 21691, 'dahaka': 11294, 'immortal': 22196, 'incarnation': 22361, 'fate': 16354, 'seeking': 39724, 'divine': 13096, 'retribution': 37561, 'embarks': 14656, 'path': 32995, 'carnage': 7743, 'mystery': 29961, 'defy': 11974, 'preordained': 34818, 'death': 11687, 'action': 1578, 'adventure': 1783, 'parkour': 32855, 'person': 33419, 'great': 19403, 'soundtrack': 41920, 'singleplayer': 40866, 'platformer': 34074, 'travel': 46343, 'atmospheric': 3998, 'classic': 8913, 'hack': 19962, 'slash': 41166, 'manipulation': 27379, 'gore': 19133, 'fantasy': 16251, 'story': 43162, 'rich': 37809, 'open': 31854, 'world': 50147, 'controller': 10029, 'puzzle': 35645, 'brink': 6828, 'agents': 1984, 'change': 8254, 'monaco': 29170, 'single': 40863, 'player': 34107, 'op': 31849, 

In [184]:
feature_vector.shape

(50872, 59792)

In [185]:
from sklearn.metrics.pairwise import cosine_similarity

def batch_cosine_similarity(matrix, batch_size=1000):
    n_samples = matrix.shape[0]
    similarity_matrix = np.zeros((n_samples, n_samples))

    for start in range(0, n_samples, batch_size):
      end = min(start + batch_size, n_samples)
      batch = matrix[start:end]
      similarity_batch = cosine_similarity(batch)
      similarity_matrix[start:end, start:end] = similarity_batch

    return similarity_matrix

similarity_matrix = batch_cosine_similarity(feature_vector)

print(similarity_matrix)

[[1.         0.00423772 0.01873501 ... 0.         0.         0.        ]
 [0.00423772 1.         0.00604704 ... 0.         0.         0.        ]
 [0.01873501 0.00604704 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [186]:
similarity_matrix.shape

(50872, 50872)

## 3. Prediksi

In [187]:
input_game = input('Nama Game: ')

Nama Game: Prince of Persia


In [188]:
list_game = games_with_metadata_df['title'].tolist()
print(list_game)

['Prince of Persia: Warrior Within™', 'BRINK: Agents of Change', "Monaco: What's Yours Is Mine", 'Escape Dead Island', 'Dungeon of the ENDLESS™', 'METAL SLUG 3', 'Enclave', 'Men of War: Assault Squad 2 - Deluxe Edition upgrade', 'Hyperdimension Neptunia Re;Birth1', 'The Sum of All Fears', 'Cold Fear™', 'LEGO® Harry Potter: Years 1-4', 'Hearts of Iron 2 Complete', 'Osmos', 'Comanche 4', 'Inquisitor', 'FORM', 'Homesick', 'Borderlands 2: Headhunter 4: Wedding Day Massacre', 'Bridge Constructor', 'Angvik', 'Sniper Elite 3 - Camouflage Weapons Pack', 'Chasm', 'Dandelion - Wishes brought to you -', 'Eradicator', 'Controller Companion', 'Herald: An Interactive Period Drama - Book I & II', 'Take Command - 2nd Manassas', 'GUILTY GEAR 2 -OVERTURE-', 'Aselia the Eternal -The Spirit of Eternity Sword-', 'The House in Fata Morgana Original Soundtrack', 'Offworld Trading Company - Soundtrack DLC', 'Luckitown', 'Barony', 'Retro/Grade', 'Dreamfall Chapters', 'Meltdown', 'NOT A HERO', 'Disciples Sacred

In [189]:
import difflib

find_close_match = difflib.get_close_matches(input_game, list_game)
print(find_close_match)

['Prince of Persia®', 'Prince of Qin', 'Voice of Pripyat']


In [190]:
close_match = find_close_match[0]
print(close_match)

Prince of Persia®


In [191]:
game_index = games_with_metadata_df[games_with_metadata_df['title'] == close_match].index[0]
print(game_index)

19579


In [192]:
similarity_score = list(enumerate(similarity_matrix[game_index]))
sorted_similar_game = sorted(similarity_score, key=lambda x:x[1], reverse=True)
sorted_similar_game[0:20]

[(19579, 1.0),
 (19153, 0.4913944829071316),
 (19028, 0.14217964881149397),
 (19413, 0.11188427880513323),
 (19543, 0.11140966184285002),
 (19621, 0.10950297643576178),
 (19334, 0.10784528835999303),
 (19057, 0.10771180325362793),
 (19818, 0.10583165746542483),
 (19420, 0.10407782377471096),
 (19987, 0.1015630791729333),
 (19676, 0.1013751846643414),
 (19161, 0.10044473677347586),
 (19411, 0.09718655822852643),
 (19186, 0.09500478189016194),
 (19942, 0.0948362480640335),
 (19920, 0.09403148940747257),
 (19828, 0.091679350948467),
 (19640, 0.09142299718419447),
 (19330, 0.09125709895144381)]

In [193]:
print(f"Game yang direkomendasikan untuk {input_game}:\n")

i=1
for game in sorted_similar_game:
  index = game[0]
  title = games_with_metadata_df[games_with_metadata_df.index == index]['title'].values[0]
  if (i<=20):
    print(f'{i}. {title}')
    i+=1

Game yang direkomendasikan untuk Prince of Persia:

1. Prince of Persia®
2. Prince of Persia: The Two Thrones™
3. Lone Fungus
4. There Is No Light: Enhanced Edition
5. -SPROUT-
6. Lumote: The Mastermote Chronicles
7. Stickit
8. BLACKHOLE
9. Seven: Enhanced Edition
10. MENTAL
11. Trials Fusion™
12. Magicians And Monsters
13. Even the Ocean
14. 3000th Duel
15. The Wizards - Enhanced Edition
16. Hoa
17. Sounds of Verity
18. Vestaria Saga I: War of the Scions
19. Super Magbot
20. Roar of Revenge
